# Exercise 02

## Aim of the exercise:

We will learn to:

- use Alphafold to predict the structure of proteins
- get simple motifs by using different protein sequences
- learn about the Alphafold parameters
- perform a disruption of your protein of interest

This exercise is based on the work of [Sergey Ovchinnikov](https://github.com/sokrypton).

# Alphafold

[AlphaFold](https://deepmind.google/technologies/alphafold/), an AI system developed by DeepMind, has solved the complex protein-folding problem, allowing for almost instant and highly accurate predictions of protein structures, which are crucial for understanding cellular functions and advancing medical research. Recognized by the [Critical Assessment of protein Structure Prediction community](https://www.predictioncenter.org/), AlphaFold has [significantly](https://www.predictioncenter.org/casp14/zscores_final.cgi) expanded the availability of protein structure data through the freely accessible AlphaFold Protein Structure Database.



## Alphafold Single

to start, we will play with a simple sequence of aminoacids (polymer of Glycines) and then we will explore different conformations that a simple protein can have.

Tasks:

 1. Open [Alphafold_single](https://colab.research.google.com/github/sokrypton/af_backprop/blob/beta/examples/AlphaFold_single.ipynb). Follow the instructions in the notebook.

 2. Change the sequence of aminoacids to a different one (e.g. all Alanines, all Leucines, etc.). What is the difference in the predicted structure?

 3. Design sequences that fold into simple secondary structures (see figure below)

 4. Design sequences that fold into 4 helix bundle (take a look at e.g. 3VJF)

 5. Design sequences that fold into beta-barrel

 6. Design sequences that fold into calcium-binding protein


<img src="challenge1.png" alt="Drawing"/>

**Hint**: Use the following information to design your sequences

<img src="statistics.png" alt="Drawing"/>

## For the project

Let's put into practice what we have learned so far.
We will use Alphafold to predict the structure of a protein and then we will try to disrupt it by introducing mutations.

Tasks:
1. Go to the official [Alphafold2](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb) notebook or [ColabFold](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb), read the instructions and predict the structure of your protein its sequence as an input. **Important Note**: There is a high probability that your protein is already in the database. Look in the [AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/).  If so, download the model and respond to the following questions.

    Questions:
    - what is the confidence score of your prediction?
    - what are the most confident regions?
    - align your crystal structure with the predicted one. What is the RMSD? (To calculate the RMSD between two proteins, you can use PyMol, load the two structures and use the command `rms_cur protein1 protein2` to get the value)

2. Use Alphafold or ColabFold to find a (minimum) set of mutations to disrupt your protein. 

    Questions:
    - what are the most common disruptive mutations?
    - are there subsets in your list of mutations? Disrupting different regions or functions?

**Hint**: Use your knowledge to analyze a multiple sequence alignment (MSA). You can use [Clustal Omega](https://toolkit.tuebingen.mpg.de/tools/clustalo) or [MAFFT](https://toolkit.tuebingen.mpg.de/tools/mafft).

**Hint 2**: consider the following table of stabilizing forces in proteins

 <img src="structure_stabilizing_forces.png" alt="Drawing"/>



Happy modeling!